# Task-1 Loading datasets


In [1]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
iris = load_iris()
ion = np.genfromtxt("ionosphere.txt", delimiter=",")

# Task-2.1- Splitting iris dataset

In [2]:
X_iris_train, X_iris_test, y_iris_train, y_iris_test = train_test_split(iris['data'], iris['target'], random_state=2105)
print(X_iris_train.shape, X_iris_test.shape)
print(y_iris_test[0])

(112, 4) (38, 4)
1


# Task-2.2- Splitting ionosphere dataset

In [3]:
X_ion = np.genfromtxt("ionosphere.txt", delimiter=",", usecols=np.arange(34))
y_ion = np.genfromtxt("ionosphere.txt", delimiter=",", usecols=34, dtype='int')
X_ion_train, X_ion_test, y_ion_train, y_ion_test = train_test_split(X_ion, y_ion, random_state=2105)
print(X_ion_test.shape)

(88, 34)


# Task 3.1- Nearest Neighbour method implementation

Implementing the nearest neighbours method was relatively straightforward. It relied on the calculation of Euclidean distance between points so I created a seperate helper function to compute those distances. The main for loop iterates over every sample in the test set and sets an initial minimum distance to infinity and index of nearest neighbour to -1. The inner loop iterates over the samples in the training set and computes the distance between the test sample and the training sample. It compares one test sample with every training sample before moving to the next test sample(if there is more than one being passed to the function). Once it has the index of the training sample with the smallest distance to the test sample, it retrieves the corresponding label for that training sample and uses that as the classification for the test sample.

In [4]:
import math

def euclidean_distance(a, b):
    return np.sqrt(np.sum((a - b) ** 2))
    
def nearest_neighbor(X_train, y_train, X_test):
    #Initialise an empty list to store all nearest neighbours
    predictions = []
    for test_value in X_test:
        min_distance = math.inf
        nearest_index = -1
        #Loop through all training samples and calculate distance to each test sample.
        for i in range(X_train.shape[0]):
            distance = euclidean_distance(test_value, X_train[i])
            #If the distance is smaller than the current minimum, update it.
            if distance < min_distance:
                min_distance = distance
                nearest_index = i
        #Get the label of the training sample with the smallest distance to the test sample.
        #Use that label as the prediction for the test sample.
        predictions.append(y_train[nearest_index])
    predictions_np = np.array(predictions)
    return predictions_np

#Checking that my implementation gives the same prediction as the sci-kit learn method
new_iris = np.array([[5, 2.9, 1, 0.2]])
prediction = nearest_neighbor(X_iris_train, y_iris_train, new_iris)
print(prediction)
print(iris['target_names'][prediction])
    

[0]
['setosa']


# Task 3.2 -Iris dataset error calculations

In [5]:
prediction_all_iris = nearest_neighbor(X_iris_train, y_iris_train, X_iris_test)
print(np.mean(prediction_all_iris == y_iris_test))
num_errors_iris = np.sum(prediction_all_iris != y_iris_test)
print("NUMBER OF ERRORS in IRIS dataset classification for K=1:", num_errors_iris)
err_rate_iris = num_errors_iris/len(y_iris_test)
print("IRIS dataset ERROR RATE for K=1:", err_rate_iris)

0.9736842105263158
NUMBER OF ERRORS in IRIS dataset classification for K=1: 1
IRIS dataset ERROR RATE for K=1: 0.02631578947368421


# Task 3.2 -Ionosphere dataset error calculations

In [6]:
prediction_all_ion = nearest_neighbor(X_ion_train, y_ion_train, X_ion_test)
print(np.mean(prediction_all_ion == y_ion_test))
num_errors_ion = np.sum(prediction_all_ion != y_ion_test)
print("NUMBER OF ERRORS in IONOSPHERE dataset classification for K=1:", num_errors_ion)
err_rate_ion = num_errors_ion/len(y_ion_test)
print("IONOSPHERE dataset ERROR RATE for K=1:", err_rate_ion)

0.9090909090909091
NUMBER OF ERRORS in IONOSPHERE dataset classification for K=1: 8
IONOSPHERE dataset ERROR RATE for K=1: 0.09090909090909091


In [7]:
label_zero = X_iris_train[y_iris_train == 0]
print(label_zero.shape)
label_one = X_iris_train[y_iris_train == 1]
print(label_one.shape)
label_two = X_iris_train[y_iris_train == 2]
print(label_two.shape)

(39, 4)
(37, 4)
(36, 4)


# Conformal Predictor Implementation Part 1- Conformity Scores

Implementing the conformal predictor was challenging. At first I tried splitting the training set into three classes, one for each corresponding label in the iris dataset, but quickly realised this would make copmutation very long and confusing. First I decided to calcualte the conformity scores. This just required the training data and the labels, since we can simply augment the training data with the test samples before calling this method. The outer loop iterates over all samples in the training set, and the inner loop iterates over all remaining samples in the training set. The corresponding labels are then compared and if they are the same, we know they are of the same class, otherwise they are different. This was important since our conformity measure was the distance to nearest sample in a different class/distance to nearest sample in the same class.

In [8]:
def calculate_conformity_scores(X_train, y_train):
    conformity_scores = []

    for i, x_train in enumerate(X_train):
        same_class_distances = []
        diff_class_distances = []

        for j, x_compare in enumerate(X_train):
            if i != j:  # Don't compare the sample to itself
                dist = euclidean_distance(x_train, x_compare)
                if y_train[i] == y_train[j]:  # Same class
                    same_class_distances.append(dist)
                else:  # Different class
                    diff_class_distances.append(dist)

        # Calculate minimum nearest distances
        nearest_same_class = min(same_class_distances)
        nearest_diff_class = min(diff_class_distances)

        # Compute the conformity score
        if nearest_same_class > 0:
            conformity_score = nearest_diff_class / nearest_same_class
        else:
            conformity_score = np.inf  # Handle division by zero case

        conformity_scores.append(conformity_score)

    return np.array(conformity_scores)
print(X_iris_test[10])
print(y_iris_train)

[5.4 3.9 1.7 0.4]
[1 0 2 1 2 0 0 2 2 1 0 0 1 1 0 2 1 2 0 0 1 1 2 1 2 2 1 2 0 0 0 2 1 1 2 0 2
 0 2 2 0 0 2 0 0 1 2 1 1 1 2 2 2 1 0 1 1 1 0 2 2 1 0 0 2 2 1 1 1 1 1 0 2 0
 1 0 2 0 2 0 0 1 1 0 1 0 2 1 0 0 1 0 0 1 0 2 0 2 0 1 0 2 0 1 0 2 1 2 2 1 2
 2]


# Conformity Score and rank check

In the following two cells I manually augmented the training set with different samples from the test set and then worked out all the conformity scores and printed them out. Since I was appending the test sample to the end, the last element was always the conformity score for the test sample. The training set had size 112 initially but became 113 with the augmentation as expected. To compute the rank, I used the method shown in the lab3 worksheet.

In [9]:
augmented_X = np.append(X_iris_train, [X_iris_test[3]], axis=0)
new_label = 0
augmented_y = np.append(y_iris_train, new_label)
conf_scores = calculate_conformity_scores(augmented_X, augmented_y)
print(conf_scores)
print(len(conf_scores))

[ 1.26929552  9.57427108  1.03390789  1.94935887  3.06819053 14.89966443
  6.51920241  4.84464532  5.09901951  3.31662479 15.06651917  6.74536878
  0.97182532  2.56347978 10.81665383  4.72581563  2.17944947  3.19374388
  6.96419414  7.18438478  2.67261242  2.15165741  2.35796522  3.70809924
  2.88675135  3.41565026  4.2031734   3.36650165 15.13274595 14.45683229
  5.464169    5.34679673  5.83095189  2.39791576  0.80659929  3.17441697
  3.44963766 14.17744688  3.02884682  2.07019668  4.46196043  7.31436942
  1.60727513 14.33527119 19.6977156   4.09267639  3.60555128  2.84604989
  1.4596009   4.5         1.55456318  2.64575131  4.96655481  6.40312424
 11.03026141  5.95818764  2.64575131  2.92326094 12.68857754  1.73205081
  0.4662524   6.59545298 13.52774926  9.23038461  0.747545    5.35412613
  4.4907312   1.96638416  2.29128785  5.09901951  4.6291005   6.46291194
  2.7774603  13.3041347   4.38748219 14.93318452  1.10194633  4.5669621
  2.96954236  8.6986589  20.49390153  9.8488578   9.

In [10]:
count = 0
for n in range(conf_scores.size):
    if 14.47411483 >= conf_scores[n]:
        count = count + 1
print(count)


105


# Conformal Predictor Implementation Part 2- Rank calculation
Manually changing the label for every sample in the training set would have taken far too long and been very inefficient, especially since there are 114 p-values for the iris dataset. 38 samples in the test set and 3 possible labels for each. The outer loop iterates over every sample in the test set. The inner loop iterates from 0 to 2, since these are the only possible labels for the iris dataset and augments the training set with each test sample and a postulated label. It then calculates the conformity scores for all elements in the augmented training set and the rank of the test sample for the postulated label. Then it moves onto the next label. Once all labels have been used, the outer loop progresses to the next sample in the test set and repeats the process. The print statements present each test sample index with the postulated labels and their corresponding p-values.
# Observation
One thing I noticed was that for quite a few samples in the test set, despite having the correct label, the p-value was very low. I suspect this could be due to the fact that the iris dataset has some overlap with features. It looks as if we have a new family (or, otherwise, the test sample is an unusual representative of an old family). A similar thing was seen with the ionosphere dataset as well. 

In [11]:
import numpy as np

def calculate_p_values_iris(X_train, y_train, X_test):
    # List to store results
    results = []
    
    # Iterate over every sample in the test set
    for test_index in range(X_test.shape[0]):
        # Iterate over each possible label
        for new_label in range(3): #because labels are 0, 1, and 2
            # Create the augmented dataset
            augmented_X = np.append(X_train, [X_test[test_index]], axis=0)
            augmented_y = np.append(y_train, new_label)
            
            # Calculate conformity scores
            conf_scores = calculate_conformity_scores(augmented_X, augmented_y)
            sample_conformity_score = conf_scores[-1]  # Conformity score for the test sample
            
            # Calculate the rank and p-value
            count = np.sum(sample_conformity_score >= conf_scores)
            p_value = count / conf_scores.size  # p-value based on rank
            
            # Store the result
            results.append((test_index, new_label, p_value))
    
    # Print results in a formatted table
    print(f"{'Test Sample Index':<20}{'Assigned Label':<15}{'P-Value':<10}")
    print("=" * 50)
    for test_index, new_label, p_value in results:
        print(f"{test_index:<20}{new_label:<15}{p_value:<10.4f}")
    
    return results
iris_results = calculate_p_values_iris(X_iris_train, y_iris_train, X_iris_test);#semi colon just so that the jupyter notebook doesnt return the entire results array again in the output cell

Test Sample Index   Assigned Label P-Value   
0                   0              0.0088    
0                   1              0.4779    
0                   2              0.0088    
1                   0              0.0088    
1                   1              0.3982    
1                   2              0.0088    
2                   0              0.0088    
2                   1              0.0088    
2                   2              0.6372    
3                   0              0.9292    
3                   1              0.0088    
3                   2              0.0088    
4                   0              0.9115    
4                   1              0.0088    
4                   2              0.0088    
5                   0              0.0088    
5                   1              0.0177    
5                   2              0.1681    
6                   0              0.0088    
6                   1              0.0088    
6                   2             

In [12]:
print(X_ion_train[0])
print(y_ion_test[3])

[ 1.       0.       0.10135  0.10811  0.       0.       0.       0.
  0.5473   0.82432  0.31081  1.       0.       0.       0.       0.
  0.37162 -1.       0.33108 -1.       0.       0.       0.       0.
 -0.42568 -1.       1.      -1.       0.55405 -0.23649  0.       0.
  0.       0.     ]
1


In [13]:
augmented_X_ion = np.append(X_ion_train, [X_ion_test[4]], axis=0)
new_label = 1
augmented_y_ion = np.append(y_ion_train, new_label)
conf_scores_ion = calculate_conformity_scores(augmented_X_ion, augmented_y_ion)
print(conf_scores_ion)
print(len(conf_scores_ion))

count = 0
for n in range(conf_scores_ion.size):
    if 3.26779199 >= conf_scores_ion[n]:
        count = count + 1
print(count)

[ 1.3535039   3.1239572   1.05306686  1.60089926  5.4719808   1.33934789
  1.56756792  6.57309437  1.04112152  0.73335741  0.97491524  3.23808741
  1.49547028  5.48156191  0.84463145  4.38693576  1.27192332  3.62677751
  5.66320976  2.09233346  1.06827153  1.42408054  6.15247926  1.09784014
  2.07640678  1.24605642  0.91549079  2.50245173  5.69623425  1.03376356
  3.50347084  1.86063347  1.07581715  1.13646146  1.04193525  6.25605378
  1.02865877  5.50523436  1.04788968  1.10647298  2.49862157  2.29965943
  1.02705097  3.81362869  1.36462875  1.01197354  1.36927023  1.73141591
  3.09907651  1.00190621  1.93831449  5.46297526  2.11750095  3.04708553
  2.88534212  1.11680421  3.49374241  0.88263185  1.18539136  0.67051516
  0.99855881  1.42293802  3.49740256  2.69419464  0.77444314  2.16102575
  1.07417312  5.00703284  1.87761573  2.01715801  3.62348126  4.4763277
  1.55910876  1.90394474  4.6319441   4.89663689  1.72620517  1.08552656
  0.99301086  5.72188928  1.3334873   5.85973626  2.

In [14]:
import numpy as np

def calculate_p_values_ion(X_train, y_train, X_test):
    # List to store results
    results = []
    
    # Iterate over every sample in the test set
    for test_index in range(X_test.shape[0]):
        # Iterate over each possible label
        for new_label in [1,-1]: #because labels are only 1 and -1
            # Create the augmented dataset
            augmented_X = np.append(X_train, [X_test[test_index]], axis=0)
            augmented_y = np.append(y_train, new_label)
            
            # Calculate conformity scores
            conf_scores = calculate_conformity_scores(augmented_X, augmented_y)
            sample_conformity_score = conf_scores[-1]  # Conformity score for the test sample
            
            # Calculate the rank and p-value
            count = np.sum(sample_conformity_score >= conf_scores)
            p_value = count / conf_scores.size  # p-value based on rank
            
            # Store the result
            results.append((test_index, new_label, p_value))
    
    # Print results in a formatted table
    print(f"{'Test Sample Index':<20}{'Assigned Label':<15}{'P-Value':<10}")
    print("=" * 50)
    for test_index, new_label, p_value in results:
        print(f"{test_index:<20}{new_label:<15}{p_value:<10.4f}")
    
    return results
ion_results = calculate_p_values_ion(X_ion_train, y_ion_train, X_ion_test);

Test Sample Index   Assigned Label P-Value   
0                   1              0.2955    
0                   -1             0.0644    
1                   1              0.8258    
1                   -1             0.0038    
2                   1              0.4205    
2                   -1             0.0265    
3                   1              0.4205    
3                   -1             0.0303    
4                   1              0.7045    
4                   -1             0.0038    
5                   1              0.2955    
5                   -1             0.0568    
6                   1              0.7614    
6                   -1             0.0038    
7                   1              0.9735    
7                   -1             0.0038    
8                   1              0.1326    
8                   -1             0.1326    
9                   1              0.5265    
9                   -1             0.0038    
10                  1             

# Average False p-values method

In [15]:
def average_false_p_value(results, y_test):
    false_p_values = []
    for test_index, new_label, p_value in results:
        true_label = y_test[test_index]  # Get the true label for the test sample
        if new_label != true_label:  # If the new label is not the true label
            false_p_values.append(p_value)  # Collect the p-value

    # Calculate the average false p-value
    return np.mean(false_p_values) if false_p_values else np.nan

# Average False p-values (iris)

In [16]:
average_false_p = average_false_p_value(iris_results, y_iris_test)
print("AVERAGE FALSE P-Value for IRIS DATASET:", average_false_p)

AVERAGE FALSE P-Value for IRIS DATASET: 0.012342803912435956


# Average False p-values (ionosphere)

In [17]:
average_false_p = average_false_p_value(ion_results, y_ion_test)
print("AVERAGE FALSE P-Value for IONOSPHERE DATASET:", average_false_p)

AVERAGE FALSE P-Value for IONOSPHERE DATASET: 0.04709022038567494


# Closing comments

## Efficiency of conformal predictor
My implementation of the conformal predictor was far from optimal. While the calculation of conformity scores was relatively fast, the computation of the rank and p-values, particularly for the ionosphere dataset took a very long time This is because there are 34 features and computing the distance between two points means calculating the sum of the square differences for 68 values at a time. For datasets larger than ionosphere it would be completely infeasible, although I am unsure what the most optimal implementation would be.